In [ ]:
# to make plot interactive  
%matplotlib qt

# importing required libraries 
from mpl_toolkits.mplot3d import Axes3D 
from pylab import *
import scipy.io
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import mat73
import random
import time

from tqdm.notebook import tqdm, trange
from ipywidgets import FloatProgress

from time import sleep

In [ ]:
def PCA(means, m):
    '''
    means: An array that contains the mean traces of the labels
    m: The total points of interest which are compressed
    
    Returns PCA of the traces
    '''
    t_mean = np.mean(means, axis=0)
    means = means - t_mean
    
    B = np.dot(means.T, means) / len(means)
    U, _, _ = np.linalg.svd(B)
    return U[:,0:m]

In [ ]:
def matloader(file_name, newer_mat=False, description=False, file_path=''):
    '''
    file_name: The name of the file that you're trying to load without the .mat extension.
    newer_mat: If the files were saved using a version of matlab that is >= 7.3 use this.
    description: If true then it will print a brief description of the loaded file.
    file_path: The path where the file is stored
    
    Returns the loaded matlab data
    '''
    directory = file_path + file_name + '.mat'
    
    if description:
        print(scipy.io.whosmat(directory))
    
    if newer_mat:
        return mat73.loadmat(directory)
    else:
        return scipy.io.loadmat(directory)

In [ ]:
def operation_sorter(labels, dataset, templating=False, reducing=[], size=100):
    '''
    labels: An array that contains all the labels that correspond to the traces in 'dataset'
    dataset: An array the contains all the traces
    templating: Binary value that creates templates for all the labels
    reducing: If the reduction matrix is inserted then the sorted data is reduced
    size: An integer that must correspond to the points of interest
    
    Returns the best matches and potentially creates a reduced template of that match
    '''
    # Initialises the template matrix
    if templating:
        templates = np.zeros([256,size])
        
    sorted_sets = []

    # Finds indexes of the best matches
    for i in range(256):
        idx = np.where(labels==i)[0]
        data_i = dataset[idx]
        
        if reducing != []:
            data_i = np.dot(data_i, reducing)
        
        sorted_sets.append(data_i)
        
        if templating:
            templates[i] = np.mean(data_i, axis=0)
        
        
    if templating:
        return templates, sorted_sets
    
    return sorted_sets

In [ ]:
def template_attack(templates, train_sorted, label_valtest, traces_valtest, m=1, amount=1):
    '''
    templates: An array that contains the templates that correspond to all labels
    train_sorted: An array that contains the sorted traces
    label_valtest: An array that contains all the labels for the test/validation data
    traces_valtest: An array that contains all the traces for the test/validation data
    
    Returns accuracy of the template attacks
    '''
    # Initialise variables
    pooled_cov = np.zeros((m,m))
    accuracy = 0
    attempts = 0

    # Perform pca
    Ureduces = PCA(templates, m)

    # Project the data
    templates_reduced = np.dot(templates, Ureduces)

    # Compute the inverse pooled Covariance Matrix
    for train_set in train_sorted:
        train_reduced = np.dot(np.array(train_set), Ureduces)
        train_set_reduced = np.dot(np.array(train_set), Ureduces)
        pooled_cov += np.cov(train_set_reduced.T)
    pooled_cov /= 256
    pooled_cov_inv = np.linalg.inv(pooled_cov)

    # Project the data
    valtest_reduced = operation_sorter(label_valtest, traces_valtest, reducing=Ureduces)
    
    # Perform the attack
    for operation,op in zip(valtest_reduced,range(len(valtest_reduced))):
        for i in range(0, len(operation), amount):
            attempts += 1
            
            scores = np.zeros([256,1])
            if amount > 1:
                attack_traces = operation[i:i+amount]
                for attack_trace in attack_traces:
                    for i in range(256):
                        scores[i] += sum(-.5 * np.dot(np.dot((attack_trace - templates_reduced[i]), pooled_cov_inv), (attack_trace - templates_reduced[i]).T))
            else:
                attack_trace = operation[i]
                for i in range(256):
                    scores[i] += sum(-.5 * np.dot(np.dot((attack_trace - templates_reduced[i]), pooled_cov_inv), (attack_trace - templates_reduced[i]).T))

            # If the highest score is the same as the score of the correct operation the TA is correct
            if scores[op] == max(scores):
                accuracy += 1
                
    return accuracy / attempts

In [ ]:
def heatmapmaker(templates, train_sorted, label_valtest, traces_valtest, 
                 pois = [10,20,30,40,50,60], traces_used = [1,2,4,8,16], progress=False):
    '''
    templates: An array that contains the templates that correspond to all labels
    train_sorted: An array that contains the sorted traces
    label_valtest: An array that contains all the labels for the test/validation data
    traces_valtest: An array that contains all the traces for the test/validation data
    pois: The number of Points Of Interests that are utilised
    traces_used: The total number of traces that is used to make a decision
    
    Returns a heatmap of accuracies for every Point Of Interest with the corresponding amount of traces used.
    '''
    # Initiating variables
    acc_matrix = np.zeros([len(pois),len(traces_used)])

    for current_max, j in zip(traces_used, range(len(traces_used))):
        if progress:
            print('Current traces used: ', current_max)
        for poi, i in zip(pois, range(len(pois))):
            print('Pois: ', poi)
            acc_matrix[i,j] = template_attack(templates, train_sorted, label_valtest, traces_valtest, poi, current_max)

    # creating figures
    fig = plt.figure(figsize=(10, 10)) 
    ax = fig.add_subplot(111) 

    # creating the heatmap
    plt.imshow(acc_matrix) 

    # adding title and labels
    ax.set_title("2D Heatmap of PCA TA") 
    ax.set_xlabel('Number of traces used in decision') 
    ax.set_ylabel('Points of Interest') 

    # displaying plot
    plt.show()
    
    return acc_matrix

In [ ]:
def feature_extraction(filename_traces, filename_labels, correlating=False, saving=True, progress=False):
    '''
    filename_traces: A string that corresponds to the name of a file that contains traces
    filename_labels: A string that corresponds to the name of a file that contains labels for the traces
    
    Returns the correlation feature
    '''
    # Load traces and labels
    traces = matloader(filename_traces, True, False)['traces']
    labels = matloader(filename_labels, False, False)['label']
    
    traces_sorted = operation_sorter(labels, traces)
    
    # Perform a type of correlation analysis
    if correlating:
        correlations = np.zeros([5000,100000])

        for i in range(0,256):
            if progress and i % 5 == 0:
                print(i)
            temp_trace = (traces - traces.mean(axis=0))/traces.std(axis=0)
            correlations += abs(np.dot(temp_trace.T, i)/temp_trace.shape[0])
        if saving:
            np.save('Correlations_abs', correlations)
    else:
        correlations = np.load('Correlations_abs.npy')
        plt.plot(range(len(correlations)), np.sum(correlations, axis=1))
        plt.show()
    
    return correlations

In [ ]:
def pipeline(filename_traces, filename_labels, filenames_test, filenames_labels, poi, decision_traces, 
             validating=False, progress=False, skiptest=False, iters=1, sampling=True):
    '''
    filename_traces: A string that corresponds to the name of a file that contains training traces
    filename_labels: A string that corresponds to the name of a file that contains labels for the training traces
    filenames_test: A string that corresponds to the name of a file that contains test traces
    filenames_labels: A string that corresponds to the name of a file that contains labels for the test traces
    poi: An integer that decides the number of Places of Interest to utilise
    decision_traces: An integer that corresponds to the number of traces used per label to make a decision
    validating: A logical that will immediately run heatmapmaker for testing purposes
    progress: A logical that decides whether progress will be shown through print statements
    skiptest: A logical that decides whether the pipeline will skip the test after training
    iters: An integer that corresponds to how often the code will iterate
    sampling: A logical that decides whether samples will be selected randomly from the traces
    
    Returns accuracies and stds from the attacks
    '''
    # Load the traces
    traces = matloader(filename_traces, True, False)['traces']
    
    # Perform feature extraction to remove unnecessary data
    poi_list = matloader('poi_list', False, False)['poi_list']
    poi_list = np.concatenate(poi_list).ravel()
    filtered_traces = traces[:, poi_list]
    
    if progress:
        print('Feature extraction completed.')
    
    # Load the labels
    labels = matloader(filename_labels, False, False)['label']
    
    # Split the data
    train_traces = filtered_traces[0:80000]
    train_labels = labels[0:80000]

    val_traces = filtered_traces[80000:90000]
    val_labels = labels[80000:90000]
    
    test_traces = filtered_traces[90000:100000]
    test_labels = labels[90000:100000]
    
    if progress:
        print('Dataset splitting completed.')
    
    # Create the templates and sort the data based on the labels
    templates, train_sorted = operation_sorter(train_labels, train_traces, templating=True)
    
    # Initialise list to store results
    accuracies = []
    
    # Get data for the parameters or get the first accuracy
    if validating:
        return heatmapmaker(templates, train_sorted, val_labels, val_traces, progress=True)
    else:
        if not skiptest:
            print('test acc after training: ', template_attack(templates, train_sorted, test_labels, 
                                                               test_traces, poi, decision_traces))
        
        stds = []

        for filename_test, filename_label in zip(filenames_test, filenames_labels):
            test_traces = matloader(filename_test, True, False)['traces']
            
            test_traces = test_traces[:,poi_list]
            test_labels = matloader(filename_label, False, False)['label']
            temp_accuracies = np.zeros([iters,])
            
            for i in range(iters):
                if sampling:
                    rows = random.sample(range(100000), 10000)
                    filtered_traces = test_traces[rows,:]
                    filtered_labels = test_labels[rows,:]
                else:
                    filtered_traces = test_traces
                    filtered_labels = test_labels
                temp_accuracies[i] = template_attack(templates, train_sorted, filtered_labels, 
                                                     filtered_traces, poi, decision_traces)
            
            accuracies.append(np.mean(temp_accuracies))
            stds.append(np.std(temp_accuracies))

            if progress:
                print(filename_test, ' completed: ', accuracies[-1], '.')
        return accuracies, stds

In [ ]:
def research_1(mode='validating', intermediary_vals=True, plotting=True, iterations=1):
    '''
    mode: A string that corresponds to the mode that is used for the run, currently only validating/other
    intermediary_vals: A logical that decides whether intermediary values are printed
    plotting: A logical that decides whether a plot is produced
    iterations: Integer that corresponds to the number of iterations
    
    returns List of accuracies and stds, and optionally a plot
    '''
    # Initialse necessary variables
    filename_traces = 'traces0'
    filename_labels = 'label0'
    filenames_test = ['traces1', 'traces2', 'traces3', 'traces4', 'traces5', 
                      'traces6', 'traces7', 'traces8', 'traces9', 'traces10']
    filenames_labels = ['label1', 'label2', 'label3', 'label4', 'label5', 
                        'label6', 'label7', 'label8', 'label9', 'label10']
    
    if mode != 'validating':
        mode = False
    else:
        return pipeline(filename_traces, filename_labels, filenames_test, filenames_labels, 50, 1, 
                          validating=mode, progress=intermediary_vals, skiptest=False, iters=iterations)
    
    # Run pipeline for results
    accuracies, stds = pipeline(filename_traces, filename_labels, filenames_test, filenames_labels, 50, 1, 
                          validating=mode, progress=intermediary_vals, skiptest=False, iters=iterations)
    if plotting:
        fig = plt.figure()
        plt.errorbar(range(len(accuracies)), accuracies, stds)
        plt.show()
    
    return accuracies, stds

In [ ]:
def research_2(mode='validating', intermediary_vals=True, plotting=True, iterations=1):
    '''
    mode: A string that corresponds to the mode that is used for the run, currently only validating/other
    intermediary_vals: A logical that decides whether intermediary values are printed
    plotting: A logical that decides whether a plot is produced
    iterations: Integer that corresponds to the number of iterations
    
    returns List of accuracies and stds, and optionally a plot
    '''
    # Initialse necessary variables
    filename_traces = 'traces0'
    filename_labels = 'label0'
    filenames_test = ['traces15', 'traces30', 'traces45', 'traces60', 'traces75', 'traces90', 
                      'traces105', 'traces120', 'traces135', 'traces150', 'traces165', 'traces180', 
                      'traces195', 'traces210', 'traces225', 'traces240']
    filenames_labels = ['label15', 'label30', 'label45', 'label60', 'label75', 'label90', 
                        'label105', 'label120', 'label135', 'label150', 'label165', 'label180', 
                        'label195', 'label210', 'label225', 'label240']
        
    if mode != 'validating':
        mode = False
        
    # Run pipeline for results
    accuracies, stds = pipeline(filename_traces, filename_labels, filenames_test, filenames_labels, 50, 1, 
                          validating=mode, progress=intermediary_vals, skiptest=False, iters=iterations)
    if plotting:
        fig = plt.figure()
        plt.errorbar(range(len(accuracies)), accuracies, stds)
        plt.show()
    
    return accuracies, stds

In [ ]:
def research_3(mode='validating', intermediary_vals=True, plotting=True, iterations=1):
    '''
    mode: A string that corresponds to the mode that is used for the run, currently only validating/other
    intermediary_vals: A logical that decides whether intermediary values are printed
    plotting: A logical that decides whether a plot is produced
    iterations: Integer that corresponds to the number of iterations
    
    returns List of accuracies and stds, and optionally a plot
    '''
    # Initialse necessary variables
    filename_traces = 'traces0'
    filename_labels = 'label0'
    
    filenames_test = ['traces243', 'traces244', 'traces245', 'traces246', 'traces247', 'traces248', 'traces249', 
                      'traces250', 'traces251', 'traces252', 'traces253', 'traces254']
    filenames_labels = ['label243', 'label244', 'label245', 'label246', 'label247', 'label248', 'label249', 
                        'label250', 'label251', 'label252', 'label253', 'label254']
    
    if mode != 'validating':
        mode = False
    
    # Run pipeline for results
    accuracies, stds = pipeline(filename_traces, filename_labels, filenames_test, filenames_labels, 50, 1, 
                          validating=mode, progress=intermediary_vals, skiptest=False, iters=iterations)
    print(accuracies, stds)
    
    if plotting:
        # Create the plots
        fig = plt.figure()
        plt.errorbar(range(len(accuracies)), accuracies, stds, label='Not retrained')
        plt.title('Final 10 trace files with and without retraining')
        
    # Initialse necessary variables
    filename_traces = 'traces244'
    filename_labels = 'label244'
    
    # Run pipeline for results
    accuracies_retrained, stds_retrained = pipeline(filename_traces, filename_labels, filenames_test, filenames_labels, 50, 1, 
                          validating=mode, progress=intermediary_vals, skiptest=False, iters=iterations)
    if plotting:
        plt.errorbar(range(len(accuracies_retrained)), accuracies_retrained, stds_retrained, label='Retrained')
        plt.legend()
        plt.show()
    
    return accuracies, stds, accuracies_retrained, stds_retrained

In [ ]:
def research_4(label=0):
    '''
    label: Integer ranging from 0 to 255, corresponds to the labels in the dataset
    
    returns a plot
    '''
    # Load the starting labels
    labels = matloader('label0', False, False)['label']
    
    # Load the starting traces
    traces = matloader('traces0', True, False)['traces']
    
    # Create the templates and sort the data based on the labels
    templates, traces = operation_sorter(labels, traces, templating=True, size=5000)
    
    # Create the plots
    fig = plt.figure()
    
    plt.subplot(2, 2, 1)
    selected_trace = traces[label][0]
    plt.plot(range(len(selected_trace)), selected_trace, label='trace0 single trace')
    plt.plot(range(len(templates[label])), templates[label], label='trace0 mean trace')
    plt.title('traces0 single trace vs traces0 mean trace')
    plt.xlabel("Time")
    plt.ylabel("Power Consumption")
    plt.legend()
    
    plt.subplot(2, 2, 3)
    plt.plot(range(len(selected_trace)), selected_trace, label='trace0 single trace')
    
    plt.subplot(2, 2, 4)
    plt.plot(range(len(templates[label])), templates[label], label='trace0 mean trace')
    
    # Load the final labels
    labels = matloader('label254', False, False)['label']
    
    # Load the final traces
    traces = matloader('traces254', True, False)['traces']
    
    # Create the templates and sort the data based on the labels
    templates, traces = operation_sorter(labels, traces, templating=True, size=5000)
    
    plt.subplot(2, 2, 2)
    selected_trace = traces[label][0]
    plt.plot(range(len(selected_trace)), selected_trace, label='trace254 single trace')
    plt.plot(range(len(templates[label])), templates[label], label='trace254 mean trace')
    plt.title('traces254 single trace vs traces254 mean trace')
    plt.xlabel("Time")
    plt.ylabel("Power Consumption")
    plt.legend()
    
    plt.subplot(2, 2, 3)
    plt.plot(range(len(selected_trace)), selected_trace, label='trace254 single trace')
    plt.title('traces0 single trace vs traces254 single trace')
    plt.xlabel("Time")
    plt.ylabel("Power Consumption")
    plt.legend()
    
    plt.subplot(2, 2, 4)
    plt.plot(range(len(templates[label])), templates[label], label='trace254 mean trace')
    plt.title('traces0 mean trace vs traces254 mean trace')
    plt.xlabel("Time")
    plt.ylabel("Power Consumption")
    plt.legend()
    
    plt.show()
    
    return None

In [ ]:
def research_5(mode='testing', intermediary_vals=True, plotting=True, iterations=1):
    '''
    mode: A string that corresponds to the mode that is used for the run, currently only validating/other
    intermediary_vals: A logical that decides whether intermediary values are printed
    plotting: A logical that decides whether a plot is produced
    iterations: Integer that corresponds to the number of iterations
    
    returns List of accuracies and stds, and optionally a plot
    '''
    # Initialse necessary variables
    filename_traces = 'traces0'
    filename_labels = 'label0'
    filenames_test = ['traces1', 'traces2', 'traces3', 'traces4', 'traces5', 'traces6', 
                      'traces7', 'traces8', 'traces9', 'traces10', 'traces15', 'traces30', 
                      'traces45', 'traces60', 'traces75', 'traces90', 'traces105', 'traces120', 
                      'traces135', 'traces150', 'traces165', 'traces180', 'traces195', 'traces210', 
                      'traces225', 'traces240', 'traces244', 'traces245', 'traces246', 'traces247', 
                      'traces248', 'traces249','traces250', 'traces251', 'traces252', 'traces253', 'traces254']
    
    filenames_labels = ['label1', 'label2', 'label3', 'label4', 'label5', 'label6', 
                        'label7', 'label8', 'label9', 'label10', 'label15', 'label30', 
                        'label45', 'label60', 'label75', 'label90', 'label105', 'label120', 
                        'label135', 'label150', 'label165', 'label180', 'label195', 'label210', 
                        'label225', 'label240', 'label244', 'label245', 'label246', 'label247', 
                        'label248', 'label249', 'label250', 'label251', 'label252', 'label253', 'label254']
        
    if mode != 'validating':
        mode = False
        
    # Run pipeline for results
    accuracies, stds = pipeline(filename_traces, filename_labels, filenames_test, filenames_labels, 50, 1, 
                          validating=mode, progress=intermediary_vals, skiptest=False, iters=iterations, sampling=False)
    if plotting:
        fig = plt.figure()
        plt.errorbar(range(len(accuracies)), accuracies, stds)
        plt.show()
    
    return accuracies, stds

In [ ]:
t0 = time.time()
accuracies_matrix = research_1(mode='validating', intermediary_vals=True, plotting=True, iterations=50)
t1 = time.time()

print('Time taken: ', t1-t0)

# Saves results after run
# np.save('acc_1', accuracies_1)
# np.save('stds_1', stds_1)

In [ ]:
accuracies_matrix

# creating figures
fig = plt.figure(figsize=(10, 10)) 
ax = fig.add_subplot(111) 

# creating the heatmap
plt.imshow(accuracies_matrix)

x = np.array([1,2,4,8,16]) # the grid to which your data corresponds
nx = x.shape[0]
no_labels = 5 # how many labels to see on axis x
step_x = int(nx / (no_labels - 1)) # step between consecutive labels
x_positions = np.arange(0,nx,step_x) # pixel count at label position
x_labels = x[::step_x] # labels you want to see
plt.xticks(x_positions, x_labels)

y = np.array([10,20,30,40,50,60]) # the grid to which your data corresponds
ny = y.shape[0]
no_labels = 6 # how many labels to see on axis x
step_y = int(ny / (no_labels - 1)) # step between consecutive labels
y_positions = np.arange(0,ny,step_y) # pixel count at label position
y_labels = y[::step_y] # labels you want to see
plt.yticks(y_positions, y_labels)

# ax.set_aspect(2)

# adding title and labels
ax.set_title("2D Heatmap of PCA TA") 
ax.set_xlabel('Number of traces used in decision') 
ax.set_ylabel('Points of Interest') 

# displaying plot
plt.show()

In [ ]:
t0 = time.time()
accuracies_2, stds_2 = research_2(mode='testing', intermediary_vals=True, plotting=True, iterations=50)
t1 = time.time()

print('Time taken: ', t1-t0)

# Saves results after run
# np.save('acc_2', accuracies_2)
# np.save('stds_2', stds_2)

In [ ]:
t0 = time.time()
accuracies_3, stds_3, accuracies_3_retrained, stds_3_retrained = research_3(mode='testing', intermediary_vals=True, 
                                                                            plotting=True, iterations=50)
t1 = time.time()

print('Time taken: ', t1-t0)

# Saves results after run
# np.save('acc_3', accuracies_3)
# np.save('stds_3', stds_3)
# np.save('acc_retrained_3', accuracies_retrained_3)
# np.save('stds_3_retrained', stds_retrained_3)

In [ ]:
accuracies_3_retrained = np.load('acc_retrained_3.npy')
accuracies_3_retrained = np.insert(accuracies_3_retrained,0,0.5472,axis=0)

accuracies_3 = np.load('acc_3.npy')

stds_3 = np.load('stds_3.npy')


print(len(accuracies_3))
print(accuracies_3)
print(len(stds_3))

stds_3_retrained = np.load('stds_3_retrained.npy')
stds_3_retrained = np.insert(stds_3_retrained,0,0,axis=0)
xax = range(244,255)

print(len([i for i in xax]))

(_, caps, _) = plt.errorbar(
    xax, accuracies_3, yerr=stds_3, capsize=2, label='Without Retraining')

(_, caps, _) = plt.errorbar(
    xax, accuracies_3_retrained, yerr=stds_3_retrained, capsize=2, label='With Retraining')

for cap in caps:
    cap.set_markeredgewidth(1)

plt.title('TA accuracy over time')
plt.xlabel("Time")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
for vals in range(0, 255, 15):
    t0 = time.time()
    research_4(label=vals)
    t1 = time.time()

    print('Time taken', vals, ': ', t1-t0)

In [ ]:
t0 = time.time()
acc_5, stds_5 = research_5()
t1 = time.time()

print('Time taken: ', t1-t0)

# Saves results after run
# np.save('acc_5', acc_5)
# np.save('stds_5', stds_5)

In [ ]:
# Displays a single trace

# trace = matloader('traces0', True, False)['traces'][0]
# fig = plt.figure()
# plt.plot(range(len(trace)), trace)
# plt.title('Power Trace')
# plt.xlabel("Time")
# plt.ylabel("Power Consumption")